# Trajectory Clustering with DTW

### Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

!pip install dtw-python
from dtw import *
import pickle
from pathlib import Path

# move back one directory
import os
os.chdir('..')


### Helper Functions

In [1]:
#@title Functions
# FUNCTIONS FOR TRAJECTORY CLUSTERING ANALYSIS
# CODE AUTHORED BY: SHAWHIN TALEBI

# IMPORT MODULES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

# ------------------------------------------------------------------------------
# df2array3D

# FUNCTION TO CONVERT PATIENT CLINICAL DATA STORED IN 2D PANDAS DATAFRAME TO 3D
# NUMPY ARRAY (WITH AND WITHOUT FILL)

# INPUTS
#   - df = pandas dataframe with raw patient data
#   - IDs = list-like object with unique patient IDs

# OUTPUTS
#   - data = 3D numpy array with forward/backawrd filled patient data where 1st
#   index corresponds to the patient ID, 2nd to time, and 3rd to variable
#   - hasCompletelyMissing = list with patient IDs with completely missing variables
#   - data_unfilled = 3D numpy array with unfilled (has NaNs) patient data where
#   1st index corresponds to the patient ID, 2nd to time, and 3rd to variable

# DEPENDENCIES
#   - numpy
#   - pandas
# ------------------------------------------------------------------------------

def df2array3D(df, IDs):

    # initialize 3d data array
    data = np.empty([len(IDs),8,5])
    data_unfilled = np.empty([len(IDs),8,5])

    # create list to store ID's which have completely missing variable values
    hasCompletelyMissing = []

    # intialize index
    i = 0

    # loop through patient IDs
    for ID in IDs:

        # create a temporary dataframe with data for ID
        temp_df = df[df.index==ID]

        # edge case: if time is duplicated keep first one
        if len(temp_df.hours_from_adm) != len(temp_df.hours_from_adm.unique()):
            bool = ~temp_df.hours_from_adm.duplicated()

        # initialize a temporary numpy array with data for ID
        temp_array = np.empty([8,5])
        temp_array[:] = np.NaN

        # add data to temporary numpy array with data for ID
        temp_array[temp_df.hours_from_adm, :] = temp_df.iloc[:, 1:6].to_numpy()

        # convert tempoary array to dataframe to perform backward/forward fill
        df2=pd.DataFrame(temp_array)
        df2=df2.fillna(method='ffill')
        df2=df2.fillna(method='bfill')

        # convert back to numpy array
        temp_array_filled=df2.to_numpy()

        # check for completely missing variables
        if np.sum(np.isnan(temp_array_filled))>23:

            # if completely missing add ID to list
            hasCompletelyMissing.append(ID)

        # store ID data in 3d data array
        data[i,:,:] = temp_array_filled
        data_unfilled[i,:,:] = temp_array

        # update index
        i = i + 1

    return data, hasCompletelyMissing, data_unfilled


# ------------------------------------------------------------------------------
# dropCompletelyMissing

# FUNCTION TO DROP SAMPLES WHICH HAVE VARIABLE VALUES THAT ARE COMPLETELY MISSING
# E.G. PATIENT HAS NO TEMPERATURE DATA OVER 24 HOURS

# INPUTS
#   - IDs = list-like object with unique patient IDs
#   - hasCompletelyMissing = list with patient IDs with completely missing variables
#   - data = 3D numpy array with patient data where 1st index corresponds to the
#    patient ID, 2nd to time, and 3rd to variable

# OUTPUTS
#   - X_train = 3D numpy array with patient data where 1st index corresponds to
#   the patient ID, 2nd to time, and 3rd to variable

# DEPENDENCIES
#   - numpy
# ------------------------------------------------------------------------------

def dropCompletelyMissing(IDs, hasCompletelyMissing, data):
    # get indicies correspinding to the IDs that have completely missing variable
    # values
    x,idrop,j = np.intersect1d(IDs,hasCompletelyMissing,return_indices=True)
    # define set of all ID indicies
    id_set = set(range(len(IDs)))
    # define set of ID indicies to drop
    drop_set = set(idrop)

    # take difference of 2 sets
    ikeep = id_set-drop_set

    # define training data
    X_train = data[list(ikeep),:,:]

    return X_train, ikeep

# ------------------------------------------------------------------------------
# standardize2Ddataframe
# FUNCTION TO STANDARDIZE 3D NUMPY ARRAY

# INPUTS
#   - df = pandas dataframe with patient data indexed by patient ID and 0th
#   column corresponds to time

# OUTPUTS
#   - df = pandas dataframe with patient data indexed by patient ID and 0th
#   column corresponds to time, and each column is standardized

# DEPENDENCIES
#   - numpy


# ------------------------------------------------------------------------------
def standardize2Ddataframe(df):
    df.iloc[:,1:] = (df.iloc[:,1:] - np.nanmean(df.iloc[:,1:], 0))/np.nanstd(df.iloc[:,1:], 0)

    return df


### Data Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

is_train = True

# Load Train vs Valid data
if is_train:
  df = pd.read_csv('/content/drive/My Drive/final_cohort_sepsis.csv')
else:
  df = pd.read_csv('/content/drive/My Drive/final_cohort_sepsis_validation.csv')

IDs = df.id.unique()

# restructure dataframe
df = df.drop(columns=['Unnamed: 0'])
df = df.set_index('id')

#Choose to apply standard deviation cap
apply_std_cap = False # No Cap being applied
std_devs = 6

cols = ['respiratoryrate', 'heartrate', 'systolicBP', 'diastolicBP', 'temperature']

if apply_std_cap:
  for i in cols:
    df[i] = np.where(df[i] > std_devs,std_devs, df[i])
    df[i] = np.where(df[i] < -std_devs,-std_devs, df[i])

# make copy of data frame for later analysis
df2 = df.copy()

In [ ]:
# standardize variables with respect to entire dataset
df = standardize2Ddataframe(df)


# store data in 3D array
data, hasCompletelyMissing, data_unfilled = df2array3D(df, IDs)

# define training data set
X_train, ikeep = dropCompletelyMissing(IDs, hasCompletelyMissing, data)
X_train_unfilled, ikeep = dropCompletelyMissing(IDs, hasCompletelyMissing, data_unfilled)

In [ ]:
# Examine Data
print("Original Data Shape: " + str(data.shape))
print("Prepared Data Shape: " + str(X_train.shape))
df

## DTW

In [ ]:
# define number of patients\n",
numPatients = X_train.shape[0]
    # initialize array to store patient distances\n",
dtw_distances = np.zeros((numPatients,numPatients))


# time computation
import timeit
start = timeit.default_timer() 

# compute distance between each patient using DTW
for i in range(numPatients):
  if i % 1000 == 0:
    print(i)

  for j in range(numPatients):      
        # do not compute self-distances
        if i==j:
            continue
            
        # do not compute distance between same pair twice
        if i>j:
            continue
        
        # perform dtw algorithm
        alignment = dtw(X_train[i,:,:], X_train[j,:,:], keep_internals=True, distance_only=True)
        
        # store distance in dtw_distances
        dtw_distances[i,j] = alignment.distance
        dtw_distances[j,i] = alignment.distance


# print computation time
stop = timeit.default_timer()
print('Time: ', stop - start)        
        
dtw_distances.shape

In [ ]:
# Save DTW calculated distance matrix
pd.DataFrame(dtw_distances).to_csv('/content/drive/MyDrive/Vader_Final_Cohort/DTW/dtw_distances_matrix.csv')

In [ ]:
# normalize distances
dtw_norm = dtw_distances/np.max(dtw_distances)

# transform distances into similarities
dtw_similarities = 1 - dtw_distances

dtw_norm = pd.DataFrame(dtw_norm)
dtw_norm.to_csv('/content/drive/MyDrive/Vader_Final_Cohort/DTW/distances_normalized_matrix.csv')

**Export DTW norm to Concensus Clustering Algorithm**